<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

We follow the example from [4]: nine randomly sampled young adults were
diagnosed with type II diabetes at the ages below.


In [ ]:
males = [19, 22, 16, 29, 24]
females = [20, 11, 17, 12]

We use the Mann-Whitney U test to assess whether there is a statistically
significant difference in the diagnosis age of males and females.
The null hypothesis is that the distribution of male diagnosis ages is
the same as the distribution of female diagnosis ages. We decide
that a confidence level of 95% is required to reject the null hypothesis
in favor of the alternative that the distributions are different.
Since the number of samples is very small and there are no ties in the
data, we can compare the observed test statistic against the *exact*
distribution of the test statistic under the null hypothesis.


In [ ]:
from scipy.stats import mannwhitneyu
U1, p = mannwhitneyu(males, females, method="exact")
print(U1)

17.0

`mannwhitneyu` always reports the statistic associated with the first
sample, which, in this case, is males. This agrees with $U_M = 17$
reported in [4]. The statistic associated with the second statistic
can be calculated:


In [ ]:
nx, ny = len(males), len(females)
U2 = nx*ny - U1
print(U2)

3.0

This agrees with $U_F = 3$ reported in [4]. The two-sided
*p*-value can be calculated from either statistic, and the value produced
by `mannwhitneyu` agrees with $p = 0.11$ reported in [4].


In [ ]:
print(p)

0.1111111111111111

The exact distribution of the test statistic is asymptotically normal, so
the example continues by comparing the exact *p*-value against the
*p*-value produced using the normal approximation.


In [ ]:
_, pnorm = mannwhitneyu(males, females, method="asymptotic")
print(pnorm)

0.11134688653314041

Here `mannwhitneyu`'s reported *p*-value appears to conflict with the
value $p = 0.09$ given in [4]. The reason is that [4]
does not apply the continuity correction performed by `mannwhitneyu`;
`mannwhitneyu` reduces the distance between the test statistic and the
mean $\mu = n_x n_y / 2$ by 0.5 to correct for the fact that the
discrete statistic is being compared against a continuous distribution.
Here, the $U$ statistic used is less than the mean, so we reduce
the distance by adding 0.5 in the numerator.


In [ ]:
import numpy as np
from scipy.stats import norm
U = min(U1, U2)
N = nx + ny
z = (U - nx*ny/2 + 0.5) / np.sqrt(nx*ny * (N + 1)/ 12)
p = 2 * norm.cdf(z)  # use CDF to get p-value from smaller statistic
print(p)

0.11134688653314041

If desired, we can disable the continuity correction to get a result
that agrees with that reported in [4].


In [ ]:
_, pnorm = mannwhitneyu(males, females, use_continuity=False,
                        method="asymptotic")
print(pnorm)

0.0864107329737

Regardless of whether we perform an exact or asymptotic test, the
probability of the test statistic being as extreme or more extreme by
chance exceeds 5%, so we do not consider the results statistically
significant.

Suppose that, before seeing the data, we had hypothesized that females
would tend to be diagnosed at a younger age than males.
In that case, it would be natural to provide the female ages as the
first input, and we would have performed a one-sided test using
``alternative = 'less'``: females are diagnosed at an age that is
stochastically less than that of males.


In [ ]:
res = mannwhitneyu(females, males, alternative="less", method="exact")
print(res)

MannwhitneyuResult(statistic=3.0, pvalue=0.05555555555555555)

Again, the probability of getting a sufficiently low value of the
test statistic by chance under the null hypothesis is greater than 5%,
so we do not reject the null hypothesis in favor of our alternative.

If it is reasonable to assume that the means of samples from the
populations are normally distributed, we could have used a t-test to
perform the analysis.


In [ ]:
from scipy.stats import ttest_ind
res = ttest_ind(females, males, alternative="less")
print(res)

TtestResult(statistic=-2.239334696520584,
            pvalue=0.030068441095757924,
            df=7.0)

Under this assumption, the *p*-value would be low enough to reject the
null hypothesis in favor of the alternative.